# Purpose

Download and transform open source data that can be used to predict travel times.
Each data set needs to be able to join onto the original dataset using LSOA name and have data for years 2014 and 2015.

**Ideas**

* [Number of schools from DfE](https://www.gov.uk/government/collections/statistics-school-and-pupil-numbers)
* Number of GPs and hosptials from [here](http://hefs.hscic.gov.uk/DataFiles.asp). [Postcodes](https://digital.nhs.uk/organisation-data-service/data-downloads/postcode-files).[NHS digital data downloads](https://digital.nhs.uk/organisation-data-service/data-downloads). 
* Size of road network from [DfT stats page](https://www.gov.uk/government/collections/road-network-size-and-condition)
* Public transport measures [from DfT](https://www.gov.uk/government/organisations/department-for-transport/about/statistics) (number of bus/train stations, number of bus/train journeys)
* [Population from ONS](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates) (include demographics such as children, students, pensioners and students)
* [ONS rural-urban classification](http://geoportal.statistics.gov.uk/datasets?q=RUC_2011_LSOA&sort=name)


The code below downloads(if download=True) the data sets and cleans them and joins them onto the travel time statistics. I could not find any data for rail. All data was available for 2014 and 2015, some for 2016.
Data was usually at local authority (LA) level and so mapped to a LSOA.

In [1]:
import zipfile
import os
import requests 
import shutil
import pyexcel
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 30)

# Create folder to store data in, if it doesn't already exist
if os.path.isdir('open-data') == False:
    os.mkdir('open-data')
    
def download_file(url, filepath_to_save_to):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(filepath_to_save_to, 'wb') as out_file:
            shutil.copyfileobj(r.raw, out_file)
    else: print('Download failed')

def download_unzip(url, file_to_extract, save_as):
        filepath = 'open-data/population-density.zip'
        download_file(url, 'open-data/zipfile.zip')
        with zipfile.ZipFile('open-data/zipfile.zip') as myzip:
            myzip.extract(file_to_extract,
                         path='open-data')
        os.rename('open-data/' + file_to_extract, 'open-data/' + save_as)
        os.remove('open-data/zipfile.zip')

## Geography lookups

In [2]:
def lsoa_lookup(download=False):
    url = 'http://ons.maps.arcgis.com/sharing/rest/content/items/baf6566ca08949c6bbd61ff81d9526da/data'
    file_to_extract = 'OA11_LSOA11_MSOA11_LAD11_EW_LUv2.csv'
    save_as = 'ons-geography-lsoa-codes.csv'
    if download:
        download_unzip(url, file_to_extract, save_as)
    df = pd.read_csv('open-data/' + save_as, encoding = 'latin', dtype = str)
    df_formatted = df[['LSOA11CD', 'LSOA11NM', 'LAD11CD', 'LAD11NM']].rename(columns={'LSOA11CD': 'LSOA_code',
                                                                                  'LAD11CD': 'LA_Code'})
    return(df_formatted)

In [3]:
lsoa_lookup(False).head()

,LSOA_code,LSOA11NM,LA_Code,LAD11NM
0,E01000001,City of London 001A,E09000001,City of London
1,E01000001,City of London 001A,E09000001,City of London
2,E01000001,City of London 001A,E09000001,City of London
3,E01000001,City of London 001A,E09000001,City of London
4,E01000003,City of London 001C,E09000001,City of London


## Urban rural classification
The data is published as a machine readable csv

In [4]:
def clean_urban_rural_classification():
    df = pd.read_csv('open-data/urban-rural-classification.csv')
    df_clean = df[['LSOA11CD', 'RUC11']].rename(columns={'LSOA11CD': 'LSOA_code', 'RUC11': 'urban_rural'})
    return(df_clean)

def urban_rural_classification(download=False):
    url='http://ons.maps.arcgis.com/sharing/rest/content/items/9855221596994bde8363a685cb3dd58a/data'
    filepath='open-data/urban-rural-classification.csv'
    if download:
        download_file(url, filepath)
    df = pd.read_csv(filepath)
    df_clean = df[['LSOA11CD', 'RUC11']].rename(columns={'LSOA11CD': 'LSOA_code', 'RUC11': 'urban_rural'})
    return(df_clean)


In [5]:
urban_rural_classification(download=False).head()

,LSOA_code,urban_rural
0,E01000001,Urban major conurbation
1,E01000002,Urban major conurbation
2,E01000003,Urban major conurbation
3,E01000005,Urban major conurbation
4,E01000006,Urban major conurbation


## Schools

In [74]:
def download_schools_data():
    urls = {'2015': 'https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/445755/SFR16_2015_LA_tables.xlsx',
            '2014': 'https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/406915/SFR15_2014_LA_tables_v102.xlsx',
            '2016': 'https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/533028/SFR20_2016_LA_Tables_2016.xlsx'}

    for year, url in urls.items():
        filepath = 'open-data/schools-' + year + '.xlsx'
        download_file(url, filepath)

def extract_school_data(filepath):
    sheet = pyexcel.get_sheet(file_name=filepath,
                                sheet_name='Table 7a', 
                             start_row = 12)

    df = pd.DataFrame({
        'LA_Code': sheet.column[1], 
        'schools_nursery_LA': sheet.column[6],
        'schools_primary_LA': sheet.column[8],
        'schools_secondary_LA': sheet.column[10],
        'schools_private_LA': sheet.column[18],
        'schools_all_LA': sheet.column[20],
    })
    df_clean = df[df.LA_Code != '']
    return(df_clean)

def extract_school_data_2016(filepath):
    """The 2016 file is different to the 2014 and 2015 one"""
    sheet = pyexcel.get_sheet(file_name=filepath,
                                sheet_name='Table 7a', 
                             start_row = 12)

    df = pd.DataFrame({
        'LA_Code': sheet.column[1], 
        'schools_nursery_LA': sheet.column[4],
        'schools_primary_LA': sheet.column[6],
        'schools_secondary_LA': sheet.column[8],
        'schools_private_LA': sheet.column[16],
        'schools_all_LA': sheet.column[18],
    })
    df_clean = df[df.LA_Code != '']
    return(df_clean)

def school_data(download=False):
    if download:
        download_schools_data()
    df_14 = extract_school_data('open-data/schools-2014.xlsx')
    df_15 = extract_school_data('open-data/schools-2015.xlsx')
    df_16 = extract_school_data_2016('open-data/schools-2016.xlsx')
    df_14['year'] = 2014
    df_15['year'] = 2015
    df_16['year'] = 2016
    
    df = pd.concat([df_14, df_15, df_16], axis = 0, ignore_index=True).sort_values(['year', 'LA_Code']).reset_index(drop=True)
    # Some LA codes are incorrect and actually for counties. I created a lookup file to fix this
    la_code_fix = pd.read_csv('correct-la-codes.csv')
    df = pd.merge(df, la_code_fix, how='left', on='LA_Code')
    df1 = df[df.correct_la_code.isnull()].drop('correct_la_code', axis=1)
    df2 = df[df.correct_la_code.isnull()==False].drop('LA_Code', axis=1).rename(columns={'correct_la_code': 'LA_Code'})
    df = pd.concat([df1, df2], axis=0, ignore_index=True).sort_values(['year', 'LA_Code']).reset_index(drop=True)
    # Expand each LA to have a record for each of its LSOAs
    df_lsoa = pd.merge(df, lsoa_lookup(), how='left', on='LA_Code')
    df_lsoa = df_lsoa.drop(['LSOA11NM', 'LAD11NM', 'LA_Code'], axis=1).drop_duplicates()
    # Force to numeric
    for s in ['nursery', 'primary', 'secondary', 'private', 'all']:
        colname = 'schools_' + s + '_LA'
        df_lsoa[colname] = pd.to_numeric(df_lsoa[colname]) 
    return(df_lsoa)
    


In [75]:
school_data(download=False).head()

,schools_all_LA,schools_nursery_LA,schools_primary_LA,schools_private_LA,schools_secondary_LA,year,LSOA_code
0,40,1,30,1,5,2014,E01011950
1,40,1,30,1,5,2014,E01011949
3,40,1,30,1,5,2014,E01011951
18,40,1,30,1,5,2014,E01011953
19,40,1,30,1,5,2014,E01011954


## Road lengths
The latest file published by DfT is a zip file. Inside it is a time series dating back to early 2000s

In [59]:
def road_length(download=False):
    
    if download:
        url = 'https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/610542/road-lengths-in-great-britain-2016.zip'
        file_to_extract = 'RDL0202 Total road length (kilometres) by road type and local authority in Great Britain, 2005 to 2016.ods'
        save_as = 'road-lengths.ods'
        download_unzip(url, file_to_extract, save_as)
        
    def extractor(year, last_four_columns_indicies):
        filepath = 'open-data/road-lengths.ods'
        sheet = pyexcel.get_sheet(file_name=filepath,
                                  sheet_name='RDL0202a_(' + str(year) + ')', 
                                  start_row = 7,
                                  row_limit = 206)
        df = pd.DataFrame({
            'LA_Code': sheet.column[0],
            'road_LA_trunk_length_km': sheet.column[last_four_columns_indicies[0]],
            'road_LA_principal_length_km': sheet.column[last_four_columns_indicies[1]],
            'road_LA_minor_length_km': sheet.column[last_four_columns_indicies[2]],
            'road_LA_total_length_km': sheet.column[last_four_columns_indicies[3]],
            'year': int(year)
        })
        return(df)
    
    df_2014 = extractor(2014, [20, 21, 22, 23] )
    df_2015 = extractor(2015, [22, 23, 24, 25] )
    df_2016 = extractor(2016, [20, 21, 22, 23] )
    df = pd.concat([df_2014, df_2015, df_2016], axis=0, ignore_index=True).sort_values(['year', 'LA_Code']).reset_index(drop=True)
    # Some LA codes are incorrect and actually for counties. I created a lookup file to fix this
    la_code_fix = pd.read_csv('correct-la-codes.csv')
    df = pd.merge(df, la_code_fix, how='left', on='LA_Code')
    df1 = df[df.correct_la_code.isnull()].drop('correct_la_code', axis=1)
    df2 = df[df.correct_la_code.isnull()==False].drop('LA_Code', axis=1).rename(columns={'correct_la_code': 'LA_Code'})
    df = pd.concat([df1, df2], axis=0, ignore_index=True).sort_values(['year', 'LA_Code']).reset_index(drop=True)
    # Expand each LA to have a record for each of its LSOAs
    df_lsoa = pd.merge(df, lsoa_lookup(), how='left', on='LA_Code')
    df_lsoa = df_lsoa.drop(['LSOA11NM', 'LAD11NM', 'LA_Code'], axis=1).drop_duplicates()
    return(df_lsoa)

In [60]:
road_length(download=False).head()

,road_LA_minor_length_km,road_LA_principal_length_km,road_LA_total_length_km,road_LA_trunk_length_km,year,LSOA_code
0,358.4,37.4,404.2,8.4,2014,E01011950
1,358.4,37.4,404.2,8.4,2014,E01011949
3,358.4,37.4,404.2,8.4,2014,E01011951
18,358.4,37.4,404.2,8.4,2014,E01011953
19,358.4,37.4,404.2,8.4,2014,E01011954


## Bus statistics
Look at how far buses have travelled a year by LA. Old data was found on national archives

In [68]:
def bus_travel(download=False):
    
    def download_bus_travel_stats():
        urls = {
            '2014': 'http://webarchive.nationalarchives.gov.uk/20150503071437/https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/413089/bus0208.xls',
            '2015': 'http://webarchive.nationalarchives.gov.uk/20160512013009/https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/463809/bus0208.xls',
            '2016': 'https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/560582/bus0208.ods'
        }
        for year, url in urls.items():
            download_file(url, 'open-data/bus-travel-' + year + '.' + url[-3:] )

    def extractor(filename):
        year = int(filename[21:25])
        sheet = pyexcel.get_sheet(file_name=filename,
                                          sheet_name='BUS0208b', 
                                          start_row = 8,
                                          row_limit = 120)
        df = pd.DataFrame({
            'LA_Code': sheet.column[0],
            'LA_name': sheet.column[2],
            'bus_LA_vehicle_km_travelled': sheet.column[6],
            'year': year
        })
        df = df[df.LA_name != ''].drop('LA_name', axis=1)
        return(df)
    
    if download:
        download_bus_travel_stats()
    df_2014 = extractor('open-data/bus-travel-2014.xls')
    df_2015 = extractor('open-data/bus-travel-2015.xls')
    df_2016 = extractor('open-data/bus-travel-2016.ods')
    df = pd.concat([df_2014, df_2015, df_2016], axis=0, ignore_index=True).sort_values(['year', 'LA_Code']).reset_index(drop=True)
    # Some LA codes are incorrect and actually for counties. I created a lookup file to fix this
    la_code_fix = pd.read_csv('correct-la-codes.csv')
    # Manual fix Tyne and Wear
    df.loc[df.LA_Code == 'E11000004', 'LA_Code'] = 'E11000007'
    df = pd.merge(df, la_code_fix, how='left', on='LA_Code')
    df1 = df[df.correct_la_code.isnull()].drop('correct_la_code', axis=1)
    df2 = df[df.correct_la_code.isnull()==False].drop('LA_Code', axis=1).rename(columns={'correct_la_code': 'LA_Code'})
    df = pd.concat([df1, df2], axis=0, ignore_index=True).sort_values(['year', 'LA_Code']).reset_index(drop=True)
    # Expand each LA to have a record for each of its LSOAs
    df_lsoa = pd.merge(df, lsoa_lookup(), how='left', on='LA_Code')
    df_lsoa = df_lsoa.drop(['LSOA11NM', 'LAD11NM', 'LA_Code'], axis=1).drop_duplicates()
    # Force to numeric
    df_lsoa['bus_LA_vehicle_km_travelled'] = pd.to_numeric(df_lsoa.bus_LA_vehicle_km_travelled)
    return(df_lsoa)

In [71]:
bus_travel(download=False).head()

,bus_LA_vehicle_km_travelled,year,LSOA_code
0,2.832659,2014,E01011950
1,2.832659,2014,E01011949
3,2.832659,2014,E01011951
18,2.832659,2014,E01011953
19,2.832659,2014,E01011954


## LSOA area size
Taken from a population density table

In [12]:
def area_km(download=False):
    if download:
        url = 'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareapopulationdensity/mid2015sape18dt11/sape18dt11mid2015lsoapopulationdensity.zip'
        file_to_extract = 'SAPE18DT11-mid-2015-lsoa-population-density.xls'
        save_as = 'population-density.xls'
        download_unzip(url, file_to_extract, save_as)
        
    sheet = pyexcel.get_sheet(file_name='open-data/population-density.xls',
                                              sheet_name='Mid-2015 Density', 
                                              start_row = 1,
                                              row_limit = 34755)
    df = pd.DataFrame({
        'LSOA_code': sheet.column[0],
        'area_square_km': sheet.column[3]
    })
    df = df.sort_values('LSOA_code').reset_index(drop=True)
    return(df)

In [13]:
area_km().head()

,LSOA_code,area_square_km
0,E01000001,0.1298
1,E01000002,0.2284
2,E01000003,0.0591
3,E01000005,0.1896
4,E01000006,0.1466


## LSOA population

In [14]:
def lsoa_population(download=False):
    def extractor(filename, sheet_name, year):
        d = pyexcel.get_sheet(file_name=filename, 
                              sheet_name=sheet_name,
                              name_columns_by_row=0) 
        df = pd.DataFrame(d.to_dict())
        df['year'] = int(year)
        return(df)
    
    params = [
        {
            'year': 2014,
            'url': 'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates/mid2014/rft-lsoa-unformatted-table-2014.zip',
            'file_to_extract':'SAPE17DT2-mid-2014-lsoa-syoa-estimates-unformatted.xls',
            'save_as': 'population-2014.xls',
            'sheet': 'Mid-2014 Persons'},
        {
            'year': 2015,
            'url': 'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates/mid2015sape18dt1/sape18dt2mid2015lsoasyoaestimatesunformatted.zip',
            'file_to_extract': 'SAPE18DT2-mid-2015-lsoa-syoa-estimates-unformatted.xls',
            'save_as': 'population-2015.xls',
            'sheet': 'Mid-2015 Persons'
        }
    ]

    dfs = []
    for item in params:
        if download:
            download_unzip(item['url'], item['file_to_extract'], item['save_as'])
        dfs.append(extractor('open-data/' + item['save_as'], item['sheet'], item['year']))
    df = pd.concat(dfs, axis=0, ignore_index=True).rename(columns={'Code': 'LSOA_code'}).drop(['Name', 'All Ages'], axis=1)
    df_long = pd.melt(df, id_vars = ['LSOA_code', 'year'], var_name='age', value_name='population')
    df_long['age'] = pd.to_numeric(df_long.age)
    df_long['pop_u18'] = (df_long.age < 18) * df_long.population
    df_long['pop_working_age'] = ((df_long.age >= 18) & (df_long.age < 65 )) * df_long.population
    df_long['pop_retired_age'] = (df_long.age >= 65) * df_long.population
    df_long.drop(['age', 'population'], axis=1, inplace=True)
    df_aggregated = df_long.groupby(['LSOA_code', 'year']).sum().reset_index()

    return(df_aggregated)


In [ ]:
lsoa_population(False).head()

# Putting it all together

In [76]:
def enriched_data():
    # Printing checks that now extra rows get added and observe how many new columns are added
    df = pd.read_csv('01_clean_stats.csv')
    print(df.shape)
    df = df.merge(urban_rural_classification(), how='left', on='LSOA_code')
    print(df.shape)
    df = df.merge(area_km(), how='left', on='LSOA_code')
    print(df.shape)
    df = df.merge(lsoa_population(), how='left', on=['LSOA_code', 'year'])
    print(df.shape)
    df = df.merge(road_length(), how='left', on=['LSOA_code', 'year'])
    print(df.shape)
    df = df.merge(bus_travel(), how='left', on=['LSOA_code', 'year'])
    print(df.shape)
    df = df.merge(school_data(), how='left', on=['LSOA_code', 'year'])
    print(df.shape)
    
    # London bus data is not recorded at LA level. Take the Tfl total and divide by 33 to split evenly accross LAs
    london_2016 = 487.9 / 33
    london_2015 = 485.0 /33
    london_2014 = 486.1 / 33
    f_2014 = (df.bus_LA_vehicle_km_travelled.isnull()) & (df.year == 2014)
    f_2015 = (df.bus_LA_vehicle_km_travelled.isnull()) & (df.year == 2015)
    f_2016 = (df.bus_LA_vehicle_km_travelled.isnull()) & (df.year == 2016)
    df.loc[f_2014, 'bus_LA_vehicle_km_travelled'] = london_2014
    df.loc[f_2015, 'bus_LA_vehicle_km_travelled'] = london_2015
    df.loc[f_2016, 'bus_LA_vehicle_km_travelled'] = london_2016

    return(df)

In [77]:
df = enriched_data()
df.to_csv('02-enriched-data.csv', index=False)

(1576512, 8)
(1576512, 9)
(1576512, 10)
(1576512, 13)
(1576512, 17)
(1576512, 18)
(1576512, 23)


In [78]:
df.head()

,LSOA_code,Region,LA_Code,LA_Name,year,mode,travel_time,nearest,urban_rural,area_square_km,pop_u18,pop_working_age,pop_retired_age,road_LA_minor_length_km,road_LA_principal_length_km,road_LA_total_length_km,road_LA_trunk_length_km,bus_LA_vehicle_km_travelled,schools_all_LA,schools_nursery_LA,schools_primary_LA,schools_private_LA,schools_secondary_LA
0,E01000001,London,E09000001,City of London,2014,car,6.753080,employment_centre,Urban major conurbation,0.1298,178,992,343,44.3,12.85,57.2,0.0,14.730303,5,0,1,4,0
1,E01000001,London,E09000001,City of London,2014,cycle,6.610821,employment_centre,Urban major conurbation,0.1298,178,992,343,44.3,12.85,57.2,0.0,14.730303,5,0,1,4,0
2,E01000001,London,E09000001,City of London,2014,public transport,3.648643,employment_centre,Urban major conurbation,0.1298,178,992,343,44.3,12.85,57.2,0.0,14.730303,5,0,1,4,0
3,E01000001,London,E09000001,City of London,2015,car,6.153411,employment_centre,Urban major conurbation,0.1298,213,1049,377,44.2,12.90,57.1,0.0,14.696970,5,0,1,4,0
4,E01000001,London,E09000001,City of London,2015,cycle,6.501751,employment_centre,Urban major conurbation,0.1298,213,1049,377,44.2,12.90,57.1,0.0,14.696970,5,0,1,4,0


In [79]:
df.describe()

,year,travel_time,area_square_km,pop_u18,pop_working_age,pop_retired_age,road_LA_minor_length_km,road_LA_principal_length_km,road_LA_total_length_km,road_LA_trunk_length_km,bus_LA_vehicle_km_travelled,schools_all_LA,schools_nursery_LA,schools_primary_LA,schools_private_LA,schools_secondary_LA
count,1576512.0,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06,1.576512e+06
mean,2014.5,1.356857e+01,3.966582e+00,3.542437e+02,1.013642e+03,2.930410e+02,3.084799e+03,3.171378e+02,3.489892e+03,8.795676e+01,3.576938e+01,2.609671e+02,3.935194e+00,1.823892e+02,2.549618e+01,3.490082e+01
std,0.5,1.138252e+01,1.330362e+01,1.253106e+02,2.939473e+02,1.276651e+02,2.813679e+03,2.753034e+02,3.159127e+03,9.268993e+01,3.510738e+01,1.877374e+02,5.534488e+00,1.339109e+02,2.352313e+01,2.477232e+01
min,2014.0,0.000000e+00,1.830000e-02,1.100000e+01,3.700000e+02,5.000000e+00,2.920000e+01,7.700000e+00,3.690000e+01,0.000000e+00,1.630230e-01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2014.0,7.279566e+00,2.724000e-01,2.710000e+02,8.410000e+02,2.010000e+02,7.073000e+02,7.130000e+01,7.988000e+02,1.180000e+01,1.469697e+01,1.050000e+02,1.000000e+00,6.800000e+01,8.000000e+00,1.500000e+01
50%,2014.5,9.678440e+00,4.553000e-01,3.350000e+02,9.550000e+02,2.780000e+02,2.050400e+03,2.001000e+02,2.403900e+03,5.420000e+01,1.928464e+01,1.870000e+02,2.000000e+00,1.300000e+02,2.000000e+01,2.800000e+01
75%,2015.0,1.533043e+01,1.285900e+00,4.130000e+02,1.115000e+03,3.660000e+02,4.837800e+03,5.410000e+02,5.558900e+03,1.597000e+02,4.198612e+01,3.670000e+02,5.000000e+00,2.810000e+02,3.300000e+01,4.500000e+01
max,2015.0,1.200000e+02,6.728352e+02,1.727000e+03,8.938000e+03,1.260000e+03,1.186640e+04,1.046800e+03,1.303660e+04,3.299000e+02,1.235597e+02,6.890000e+02,2.700000e+01,4.830000e+02,1.100000e+02,1.020000e+02
